# INFO 2950 Project Phase II

Ray Zhou, Ray Zhang, Alex Weiss, Steven Chen

To do:
- complete 'Summary Statistcs/Exploratory Data Analysis' section with plots and connect back to Research Question (provide analysis but don't say 'cause', just describe if there is a correlation
- summarize quickly some limiations/problems of the dataset. Please don't say that this dataset is fundamentally flawed lol, try to find issues that say we did a lot of work to clean the data
- ask some basic questions to the reviewers
- SUBMIT !!! one of you guys can submit when you are done and we can all download from the CMSX


Tips:
- use the Zoom "Remote Control" function while working on one computer, it really helps because edits are not shared real time
- group data by boroughs
    - Graph section 1: we have 5 boroughs and casualties/vehicles involved in accident. We can generate a scatterplot/line plot and analyzing correlation for a total of 10 graphs
    - Graph section 2: create a histogram comparing each borough in terms of counts of accidents to provide a holistic perspective
    
Feel free too add more or adjust as you like, these are just suggestions to meet the Phase II criteria.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
accidents = pd.read_csv('Motor_Vehicle_Collisions_-_Crashes.csv')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


---

## Research Question

What impact does the **time of day** have on **accident severity** in terms of **numbers of casualties** and **vehicles involved in an accident** in the **various boroughs of New York**?

---

## Data Description

Motor vehicle collision data was downloaded from the NYC OpenData project interface. The website can be found [here](https://data.cityofnewyork.us/Public-Safety/Motor-Vehicle-Collisions-Crashes/h9gi-nx95). Data points originate from filed MV104-AN police reports.

Since the dataset was of 1.76 million datapoints since 2012 with 29 factors, we can omit certain factors are less related to the research question. For now some important factors include time, borough, persons killed, and persons injured. The types of vehicles involved in an accident are also summarized in five columns, we will have to perform operations to generate another column to sum up the number of vehicles involved in each crash.

---

## Data Collection, and Cleaning

First we can rename some of the columns we will operate on.

In [2]:
accidents = accidents.rename(columns = {'CRASH DATE':'date','CRASH TIME':'time', 'BOROUGH':'borough', 'NUMBER OF PERSONS KILLED': 'killed', 'NUMBER OF PERSONS INJURED': 'injured'})

First we can evaluate the number of vehicles involved in an accident. In the original dataset, there are five columns. As crashes are arranged by row, the columns filled represents the types of vehicles involved, with NaN meaning a vehicle was not involved in a crash. By counting the number of NaNs in each row of the vehicle involved dataframe we can determine the total number of vehicles in a crash. We can append this to the original dataset in a new column called 'vehicles_involved'.

In [3]:
vehicle_types = ['VEHICLE TYPE CODE 1', 'VEHICLE TYPE CODE 2', 'VEHICLE TYPE CODE 3', 'VEHICLE TYPE CODE 4', 'VEHICLE TYPE CODE 5']
accident_vehicles = accidents[vehicle_types]

accidents['vehicles_involved'] = 5-accident_vehicles.isnull().sum(axis=1)



We also create a list of factors which have to deal with the research question. We want to see if the time of day impacts the severity of a crash in terms of
1. Number of casualties 
2. Number of vehicles involved

and we are analyzing this in each borough.

Using this list of factors we subset the accidents data.

In [4]:
factors = ['date', 'time', 'borough', 'killed', 'injured', 'vehicles_involved']
accidents = accidents[factors]

Next we can focus only on accidents in the incorporated boroughs. According to the data origin from New York State MV-104AN forms, accidents not given a borough (represented by NaN) occured in an nearby county (which we will omit).

In [5]:
accidents = accidents.dropna(subset = ["borough"])
accidents.head()

,date,time,borough,killed,injured,vehicles_involved
0,10/30/2020,10:00,BROOKLYN,0.0,0.0,1
3,10/09/2020,17:10,BRONX,0.0,0.0,2
4,05/03/2018,16:53,BROOKLYN,0.0,1.0,1
6,10/21/2020,13:25,MANHATTAN,0.0,0.0,2
8,05/01/2018,20:50,BROOKLYN,0.0,1.0,2


Now we generate a new column of time objects. Since we have two columns with the date and the time of day, we can concatenate into one column, and using Pandas to_datetime(), parse into time objects.

In [6]:
accidents['parsed_time'] = pd.to_datetime(accidents.date + accidents.time, format = '%m/%d/%Y%H:%M')
accidents.head()

,date,time,borough,killed,injured,vehicles_involved,parsed_time
0,10/30/2020,10:00,BROOKLYN,0.0,0.0,1,2020-10-30 10:00:00
3,10/09/2020,17:10,BRONX,0.0,0.0,2,2020-10-09 17:10:00
4,05/03/2018,16:53,BROOKLYN,0.0,1.0,1,2018-05-03 16:53:00
6,10/21/2020,13:25,MANHATTAN,0.0,0.0,2,2020-10-21 13:25:00
8,05/01/2018,20:50,BROOKLYN,0.0,1.0,2,2018-05-01 20:50:00


According to the handbook "You should be saving data in intermediate files at several points through this process so that you are not starting from scratch every time you change something". To do so we will use Pandas to_csv() method.

In [7]:
accidents.to_csv('cleaned_accidents.csv')

---

## Data Limitations


---

## Summary Statistics/Exploratory Data Analysis

In [22]:
print('{} {:.4f}'.format('Mean vehicles involved:', np.mean(accidents.vehicles_involved)))
print('{} {:.4f}'.format('Standard deviation of vehicles involved', np.std(accidents.vehicles_involved)))

print()
print(accidents.groupby(["borough"])["vehicles_involved"].mean())



Mean vehicles involved: 1.8940
Standard deviation of vehicles involved 0.5386

borough
BRONX            1.856926
BROOKLYN         1.897960
MANHATTAN        1.893819
QUEENS           1.910230
STATEN ISLAND    1.887740
Name: vehicles_involved, dtype: float64


---

## Questions for Reviewers

1. What conclusion can we come to if one borough has significantly more accident casualties/vehicles involved than another? Essentially, if you have five categorical variables, what kind of correlation conclusion can you come to? Are there any tests that can be done to compare such categorical variables?